# [VGG](https://arxiv.org/pdf/1409.1556.pdf)

Implement VGG16, for that write specific `nn.Module`, `VGGBlock` implementing block of VGG.

In [ ]:
import torch
import torch.nn as nn

class VGG16(nn.Module):
    def __init__(self, in_channels, max_features=1000):
        super().__init__()
        self.in_channels = in_channels
        self.max_features = max_features

        self.vggb1 = self.VGGBlock(2, self.in_channels, 64)
        self.vggb2 = self.VGGBlock(2, 64, 128)
        self.vggb3 = self.VGGBlock(3, 128, 256)
        self.vggb4 = self.VGGBlock(3, 256, 512)
        self.vggb5 = self.VGGBlock(3, 512, 512)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))
        self.linear = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=1000, bias=True)
        )

    def forward(self, x):
        x = self.vggb1(x)
        x = self.vggb2(x)
        x = self.vggb3(x)
        x = self.vggb4(x)
        x = self.vggb5(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.linear(x)

        return nn.Softmax(x)

    def VGGBlock(self, convs_num, in_channels, out_channels):
        res = []
        res.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1))
        res.append(nn.ReLU())
        for i in range(convs_num - 1):
            res.append(
                nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1)
            )
            res.append(nn.ReLU())
        res.append(nn.MaxPool2d(kernel_size=2, stride=2, padding=0))
        return nn.Sequential(
            *res
        )


# [GoogLeNet](https://arxiv.org/pdf/1409.4842.pdf)

## Inception module

Write specific `nn.Module` for Inception module.

In [ ]:
from torch.nn import functional as F

class Inception(nn.Module):
    # `c1`--`c4` are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

## Stem network

Write down, why do we need a Stem network.

*Write your answer here.*

It consists of three convolutional layers (the first with a large filter) with two unifying layers and is located at the very beginning of the architecture. The purpose of this subnetwork is to quickly and strongly reduce the spatial dimensions (compress the image before parallel processing) in order to minimize the number of elements in the layers.

# [ResNet](https://arxiv.org/pdf/1512.03385.pdf)

Implement ResNet-18, for that write specific `nn.Module`, `ResNetBlock` implementing block of ResNet.

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

class Residual(nn.Module):
    """The Residual block of ResNet."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.bn1 = nn.LazyBatchNorm2d()
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, x):
        Y = self.conv1(x)
        Y = self.bn1(Y)
        Y = F.relu(Y)
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            x = self.conv3(x)
        Y += x
        return F.relu(Y)


class ResNet(nn.Module):
    def __init__(self, arch, lr=0.1, num_classes=10):
        super(ResNet, self).__init__()
        self.net = nn.Sequential(self.b1())
        for i, b in enumerate(arch):
            self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i==0)))
        self.net.add_module('last', nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
            nn.LazyLinear(num_classes)))

    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
        
    def block(self, num_residuals, num_channels, first_block=False):
        blk = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                blk.append(Residual(num_channels))
        return nn.Sequential(*blk)

    def forward(self, x):
        return self.net(x)


class ResNet18(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)),
                       lr, num_classes)
        
    def forward(self, x):
        return self.net(x)

ResNet18()

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


ResNet18(
  (net): Sequential(
    (0): Sequential(
      (0): LazyConv2d(0, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (b2): Sequential(
      (0): Residual(
        (conv1): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn2): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Residual(
        (conv1): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (bn1): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): LazyConv2d(0, 64, kerne

Слегка странно, что модель строится из init, а не из forward метода, но там есть всё необходимое (в блоке).

# [ResNeXt](https://arxiv.org/pdf/1611.05431.pdf)

Write specific `nn.Module`, `ResNeXtBlock` implementing block of ResNeXt.

In [ ]:
class ResNeXtBlock(nn.Module):
    """The ResNeXt block."""
    def __init__(self, num_channels, groups, bot_mul, use_1x1conv=False,
                 strides=1):
        super().__init__()
        bot_channels = int(round(num_channels * bot_mul))
        self.conv1 = nn.LazyConv2d(bot_channels, kernel_size=1, stride=1)
        self.conv2 = nn.LazyConv2d(bot_channels, kernel_size=3,
                                   stride=strides, padding=1,
                                   groups=bot_channels//groups)
        self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1, stride=1)
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()
        self.bn3 = nn.LazyBatchNorm2d()
        if use_1x1conv:
            self.conv4 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
            self.bn4 = nn.LazyBatchNorm2d()
        else:
            self.conv4 = None

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = F.relu(self.bn2(self.conv2(Y)))
        Y = self.bn3(self.conv3(Y))
        if self.conv4:
            X = self.bn4(self.conv4(X))
        return F.relu(Y + X)

# [SENet](https://arxiv.org/pdf/1709.01507.pdf)

Write specific `nn.Module`, `SEBlock` implementing block of SENet.

In [ ]:
class SE_Block(nn.Module):
    def __init__(self, c, r=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(c, c // r, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(c // r, c, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        bs, c, _, _ = x.shape
        y = self.squeeze(x).view(bs, c)
        y = self.excitation(y).view(bs, c, 1, 1)
        return x * y.expand_as(x)

# [Neural Architecture Search](https://arxiv.org/pdf/1611.01578.pdf)

For the neural network of your assignment 2, write down the parametrization of the network you would use for the NAS.

*Write your answer here.*

Для своей сети я смогу подбирать количество слоёв, выбирать функцию активации, количество нейронов в слоях. В принципе всё, у меня не свёрточная нейронная сеть, так что вот.

In [ ]:
STATE_SIZE = 9
ACTIONS_SIZE = 4
HIDDEN_LAYER_SIZE = 2 ** STATE_SIZE


class DQN(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(STATE_SIZE, HIDDEN_LAYER_SIZE)
        self.hidden_1 = nn.Linear(HIDDEN_LAYER_SIZE, HIDDEN_LAYER_SIZE // 2)
        self.hidden_2 = nn.Linear(HIDDEN_LAYER_SIZE // 2, HIDDEN_LAYER_SIZE // 2)
        self.output = nn.Linear(HIDDEN_LAYER_SIZE // 2, ACTIONS_SIZE)

        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
    

    def forward(self, x):
        x = self.input(x)
        x = self.relu(self.hidden_1(x))
        x = self.relu(self.hidden_2(x))
        x = self.output(x)
        
        return self.softmax(x)